<a href="https://colab.research.google.com/github/tselane2110/Forecasting-Sticker-Sales/blob/main/StickerSalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e1_path = kagglehub.competition_download('playground-series-s5e1')

print('Data source import complete.')


100%|██████████| 2.26M/2.26M [00:00<00:00, 88.0MB/s]

Extracting files...
Data source import complete.


In [3]:
print(playground_series_s5e1_path)

/root/.cache/kagglehub/competitions/playground-series-s5e1


## 1. Importing required libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# getting the path to the files

import os
for dirname, _, filenames in os.walk('/root/.cache/kagglehub/competitions/playground-series-s5e1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/root/.cache/kagglehub/competitions/playground-series-s5e1/train.csv
/root/.cache/kagglehub/competitions/playground-series-s5e1/sample_submission.csv
/root/.cache/kagglehub/competitions/playground-series-s5e1/test.csv


## 2. Importing Dataset

In [6]:
df = pd.read_csv("/root/.cache/kagglehub/competitions/playground-series-s5e1/train.csv")
df.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


## 3. Understanding Data

In [10]:
df.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [11]:
df.dtypes

,0
id,int64
date,object
country,object
store,object
product,object
num_sold,float64


In [12]:
df.describe()

,id,num_sold
count,230130.000000,221259.000000
mean,115064.500000,752.527382
std,66432.953062,690.165445
min,0.000000,5.000000
25%,57532.250000,219.000000
50%,115064.500000,605.000000
75%,172596.750000,1114.000000
max,230129.000000,5939.000000


In [23]:
221259/230130

0.9614522226567592

In [14]:
len(df['date'].unique())

2557

In [21]:
2557/365

7.005479452054795

In [15]:
df['country'].unique()

array(['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore'],
      dtype=object)

In [17]:
df['product'].unique()

array(['Holographic Goose', 'Kaggle', 'Kaggle Tiers', 'Kerneler',
       'Kerneler Dark Mode'], dtype=object)

In [18]:
df['store'].unique()

array(['Discount Stickers', 'Stickers for Less', 'Premium Sticker Mart'],
      dtype=object)

In [20]:
len(df['num_sold'].unique())

4038

## 3.1 Understanding Data (Summary)

Our data has 6 columns (id, date, country, product, store, num_sold). We realized that:
1. The `id` column is not required, hence, we will delete it during the **data-cleaning** phase.
2. The <u>datatype</u> of the `date` column is "object" which we will change it to "datetime" during the **data-cleaning** phase.

### About `date`:
We checked the total **unique** values in our date column which turned out to be 2557. Assuming that every-year has 365 days, we divided 2557 by 365 and got 7.005 as an answer. This means that we have about around 7 years of data. However, we will still have to fact-check by understanding our data more during the **data-cleaning** phase.

### About the Variance in the data:
Since the target-column is `num_sold`, we checked the **mean** and **standard-deviation** of this column to understand the distribution of the data. The mean we got is 752.527 whereas the standard-deviation (std) is around 690.165. The minimum value in the `num_sold` is 5 whereas the maximum is 5939. This might mean that our data has high-variance (not too high but still high) and there might be some outliers (some stores might be selling some products more than the other stores) which we will further explore in the **data-cleaning** phase.

### About missing values
We saw that the total number of rows in the dataset are 230130 whereas the total number of "present values" in the `num_sold` column is 221259. This implies that (for `num_sold` column) 4% of the data is missing. However, we will further explore the missing values of all columns in the **data-cleaning** phase.

## 4. Data Cleaning

In [26]:
# fixing the datatype of the 'date' column
df['date'] = pd.to_datetime(df['date'])

In [28]:
df.dtypes

,0
id,int64
date,datetime64[ns]
country,object
store,object
product,object
num_sold,float64


In [29]:
df['date'][0]

Timestamp('2010-01-01 00:00:00')

In [30]:
df.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [37]:
# looking for the count of missing values in each column
date = df['date'].isna().sum()
country = df['country'].isna().sum()
store = df['store'].isna().sum()
product = df['product'].isna().sum()
sales = df['num_sold'].isna().sum()

print(f"""Missing Values for all columns are:\n
          For date column: {date}\n
          For country column: {country}\n
          For store column: {store}\n
          For product column: {product}\n
          For num_sold column: {sales}""")

Missing Values for all columns are:

          For date column: 0

          For country column: 0

          For store column: 0

          For product column: 0

          For num_sold column: 8871


In [40]:
# looking for duplicate rows
print(f"We have {df.duplicated().sum()} duplicate rows in our dataset.")

We have 0 duplicate rows in our dataset.
